In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

/kaggle/input/newdataset/vk.mp3


## ⚙️ Install Required Libraries

This installs `whisper`, `google-generativeai`, and [langgraph](https://www.langgraph.dev/) — a library for building multi-step AI workflows.  
Run this cell only once per session.


In [2]:
# Install dependencies (run only once)
!pip install -q openai-whisper google-generativeai langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 13.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/

In [3]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.


In [4]:
!pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 51.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 77.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 80.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 50.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 87.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 3.3 MB/

## 📦 Import Libraries & Setup Gemini Model

Imports `whisper` for speech-to-text, `genai` for Gemini, and `langgraph` for workflow logic.  
Initializes Gemini 1.5 Pro for AI content generation.


In [5]:
# Import necessary libraries
import whisper
import google.generativeai as genai
from langgraph.graph import StateGraph, END
import json
import os

gemini_model = genai.GenerativeModel('gemini-1.5-pro')

In [6]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [7]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai

# Load API key from Kaggle secrets
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Configure Gemini client
genai.configure(api_key=GOOGLE_API_KEY)


## 🧠 Load Whisper ASR Model

This loads the `"small"` Whisper model by OpenAI for converting audio to text.  
🔗 [Whisper GitHub](https://github.com/openai/whisper) | Great for speech recognition tasks.


In [8]:
# Load Whisper ASR (Automatic Speech Recognition) model
whisper_model = whisper.load_model("small")

def transcribe_audio(audio_file_path):
    result = whisper_model.transcribe(audio_file_path)
    text = result["text"]
    print(text)  # <-- print inside the function
    return text

100%|███████████████████████████████████████| 461M/461M [00:14<00:00, 33.1MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

##  Few-Shot Grammar Correction Examples

Provides sample input-output pairs to guide Gemini in correcting grammar.  
Helps the model understand the expected correction format for similar sentences.


In [9]:
# Few-shot examples for better corrections
few_shot_examples = """
Example 1:
User: i am working in Google
Response: {"original": "i am working in Google", "corrected": "I am working at Google."}

Example 2:
User: she go to school every day
Response: {"original": "she go to school every day", "corrected": "She goes to school every day."}

Example 3:
User: they is playing outside
Response: {"original": "they is playing outside", "corrected": "They are playing outside."}
"""

## 🤖 Correct Grammar Using Gemini with JSON Output

This function sends a prompt to [Gemini 1.5 Pro](https://ai.google.dev/) to fix grammar mistakes in a sentence.  
It uses few-shot examples to help Gemini understand the correction style.  
The model's response is expected in structured JSON like `{"original": ..., "corrected": ...}`.  
If the output isn't valid JSON, it gracefully falls back to a default format.  
This ensures consistent and clean correction results that can be easily parsed and reused.


In [10]:
# Correct text using Gemini and structured JSON output
def correct_text_with_gemini(text):
    prompt = f"""
You are an expert English grammar corrector. Always respond with a valid JSON object.

{few_shot_examples}

Now correct this sentence:
User: {text}
Response:
"""
    response = gemini_model.generate_content(prompt)

    try:
        # Ensure valid JSON format
        correction_json = json.loads(response.text)
    except json.JSONDecodeError:
        correction_json = {"original": text, "corrected": response.text.strip()}

    return correction_json

## 🔄 Define LangGraph State and Nodes

This code defines a custom state `AudioCorrectionState` using Python's `@dataclass`, which holds:
- `audio_path`: the location of the audio file.
- `text`: transcribed text from the audio.
- `corrected`: dictionary storing both original and corrected sentences.

Two **LangGraph nodes** (functions) are defined:
- `node_transcribe`: uses Whisper to convert audio into text.
- `node_correct`: uses Gemini to fix grammar in the transcribed text.

🧠 Learn more about LangGraph stateful workflows:  
🔗 [LangGraph GitHub](https://github.com/langchain-ai/langgraph)


In [11]:
# Define the LangGraph State
from dataclasses import dataclass

@dataclass
class AudioCorrectionState:
    audio_path: str = ""
    text: str = ""
    corrected: dict = None

def node_transcribe(state):
    audio_path = state.audio_path   # <-- FIXED
    text = transcribe_audio(audio_path)
    state.text = text               # <-- FIXED
    return state

def node_correct(state):
    text = state.text               # <-- FIXED
    corrected = correct_text_with_gemini(text)
    state.corrected = corrected     # <-- FIXED
    return state

## 🧩 Create LangGraph Workflow

This builds a **LangGraph workflow** using the `AudioCorrectionState` as the state container.  
- `add_node()` adds processing steps (`transcribe` and `correct`) to the graph.  
- `set_entry_point("transcribe")` sets the workflow's starting point.  
- `add_edge()` defines the execution flow: transcribe ➝ correct ➝ END.  
Finally, `graph.compile()` builds the ready-to-run app from this logic.

🔗 Learn more at [LangGraph Documentation](https://docs.langgraph.dev/)


In [12]:
# Create LangGraph Workflow
graph = StateGraph(AudioCorrectionState)
graph.add_node("transcribe", node_transcribe)
graph.add_node("correct", node_correct)

graph.set_entry_point("transcribe")
graph.add_edge("transcribe", "correct")
graph.add_edge("correct", END)

app = graph.compile()

## 🧠 Step-by-step Audio Correction & Speech Synthesis using Whisper + Gemini + TTS

1. **Audio Input**: Load an audio file (e.g., `vk.mp3`) to process.
2. **Invoke LangGraph App**: Automatically runs transcription (Whisper) and grammar correction (Gemini 1.5).
3. **Extract Corrected Text**: Parse the returned result to fetch only the final corrected sentence.
4. **Initialize TTS Model**: Use 🤖 [Coqui TTS](https://github.com/coqui-ai/TTS) to synthesize speech in the original speaker's voice.
5. **Speech Generation**: Output corrected audio as `corrected_audio.mp3` using the original speaker's tone (via `speaker_wav`).

✅ This creates a full speech-to-corrected-speech loop: **MP3 ➝ Clean Text ➝ Corrected MP3**

🔗 TTS Docs: [https://tts.readthedocs.io](https://tts.readthedocs.io)


In [13]:
from TTS.api import TTS
import json
import re

# Step 1: Audio file path
audio_file_path = "/kaggle/input/newdataset/vk.mp3"

# Step 2: Prepare input
inputs = {
    "audio_path": audio_file_path,
}

# Step 3: Run transcription and correction
result = app.invoke(inputs)

# Step 4: Extract corrected text
corrected_raw = result['corrected']

# Step 5: If corrected_raw is a dict, get 'corrected' field
if isinstance(corrected_raw, dict):
    corrected_raw = corrected_raw.get("corrected", "").strip()
else:
    raise ValueError("Unexpected format: corrected_raw should be a dict!")

# Step 6: Clean up the triple backticks and JSON identifier
corrected_raw = corrected_raw.replace("```json", "").replace("```", "").strip()

# Step 7: Parse it into JSON
corrected_dict = json.loads(corrected_raw)

# Step 8: Now extract ONLY the 'corrected' text
corrected_text = corrected_dict.get("corrected", "").strip()

# Step 9: Print only corrected sentence
print("\nCorrected Text (final to synthesize):", corrected_text)

# Step 10: Prepend a label if needed
corrected_text = "corrected Text is " + corrected_text

# Step 11: Initialize TTS model
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=True, gpu=False)

# Step 12: Generate corrected audio
if corrected_text:
    tts.tts_to_file(
        text=corrected_text,
        file_path="corrected_audio.mp3",
        speaker_wav=audio_file_path,  # Speaker voice sample
        language="en"
    )
    print("\n✅ Corrected audio saved as corrected_audio.mp3")
else:
    print("\n⚠️ No corrected text found!")


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 She go to school every day but not have books. Yesterday he ate apple and drink milk but he don't like it. They was playing in a park but then rains came. I no understand why people is not helps each other. Me very tired because walking too many kilometers.

Corrected Text (final to synthesize): She goes to school every day but doesn't have books. Yesterday he ate an apple and drank milk, but he didn't like it. They were playing in a park, but then rain came. I don't understand why people don't help each other. I'm very tired because I walked too many kilometers.
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts


 98%|█████████▊| 419M/425M [00:04<00:00, 85.1MiB/s]

 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


/usr/local/lib/python3.11/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:512
 | > power:1.5
 | > preemphasis:0.97
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:False
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:False
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:True
 | > db_level:-27.0
 | > stats_path:None
 | > base:10
 | > hop_length:160
 | > win_length:400
 > External Speaker Encoder Loaded !!
 > initialization of language-embedding layers.
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10